In [1]:
# # Debugging ONLY! autoreloads when scripts update
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import contextily as ctx

import os

from src import preprocessing, config

# Obtain raw data

In [1]:
# Get 2020 Decennial Census data
decennial2020_dp_raw = preprocessing.fetch_2020_demographic_profile()

NameError: name 'preprocessing' is not defined

In [ ]:
np.array(decennial2020_dp_raw).shape

(2328, 646)

# Perform initial cleaning

In [2]:
# Clean NYC census tract shapefiles
gdf = preprocessing.clean_tracts(config.TRACTS_RAW, config.TRACTS_CLEAN)

gdf.sample(n=5)

,BOROUGH,TRACT,AREA,LAT,LONG,geometry
GEOID,,,,,,
36005016600,The Bronx,166,285689,40.829528,-73.830183,"POLYGON ((-73.8363 40.82837, -73.83611 40.8284..."
36085027301,Staten Island,273.01,3029804,40.599683,-74.152143,"POLYGON ((-74.16329 40.60168, -74.16248 40.604..."
36047038100,Brooklyn,381,239524,40.681048,-73.925426,"POLYGON ((-73.92865 40.67975, -73.92864 40.679..."
36005018400,The Bronx,184,447412,40.837416,-73.829281,"POLYGON ((-73.83501 40.83811, -73.83327 40.838..."
36061020500,Manhattan,205,281121,40.810579,-73.965139,"POLYGON ((-73.97349 40.81307, -73.97162 40.815..."


In [5]:
gdf.shape

(2327, 6)

In [3]:
decennial2020_dp_clean = preprocessing.clean_2020_demographic_profile()

decennial2020_dp_clean.sample(n=5)

,pop,pop_under5,pop_5to9,pop_10to14,pop_15to19,pop_20to24,pop_25to29,pop_30to34,pop_35to39,pop_40to44,pop_45to49,pop_50to54,pop_55to59,pop_60to64,pop_65to69,pop_70to74,pop_75to79,pop_80to84,pop_85plus,pop_16plus,pop_18plus,pop_21plus,pop_62plus,pop_65plus,pop_male,pop_male_under5,pop_male_5to9,pop_male_10to14,pop_male_15to19,pop_male_20to24,pop_male_25to29,pop_male_30to34,pop_male_35to39,pop_male_40to44,pop_male_45to49,pop_male_50to54,pop_male_55to59,pop_male_60to64,pop_male_65to69,pop_male_70to74,pop_male_75to79,pop_male_80to84,pop_male_85plus,pop_male_16plus,pop_male_18plus,pop_male_21plus,pop_male_62plus,pop_male_65plus,pop_female,pop_female_under5,pop_female_5to9,pop_female_10to14,pop_female_15to19,pop_female_20to24,pop_female_25to29,pop_female_30to34,pop_female_35to39,pop_female_40to44,pop_female_45to49,pop_female_50to54,pop_female_55to59,pop_female_60to64,pop_female_65to69,pop_female_70to74,pop_female_75to79,pop_female_80to84,pop_female_85plus,pop_female_16plus,pop_female_18plus,pop_female_21plus,pop_female_62plus,pop_female_65plus,medianage,medianage_male,medianage_female,pop_onerace,pop_white,pop_black,pop_nativeam,pop_asian,pop_hipacific,pop_other,pop_multi,racetally,racetally_white,racetally_black,racetally_nativeam,racetally_asian,racetally_hipacific,racetally_other,pop_hispanic,pop_nonhispanic,pop_hispanic_white,pop_hispanic_black,pop_hispanic_nativeam,pop_hispanic_asian,pop_hispanic_hipacific,pop_hispanic_other,pop_hispanic_multi,pop_nonhispanic_white,pop_nonhispanic_black,pop_nonhispanic_nativeam,pop_nonhispanic_asian,pop_nonhispanic_hipacific,pop_nonhispanic_other,pop_nonhispanic_multi,pop_household,pop_household_householder,pop_household_spouse_oppsex,pop_household_spouse_samesex,pop_household_unmpartner_oppsex,pop_household_unmpartner_samesex,pop_household_child,pop_household_child_under18,pop_household_grandchild,pop_household_grandchild_under18,pop_household_otherrelatives,pop_household_nonrelatives,pop_gpquarters,pop_gpquarters_institutionalized,pop_gpquarters_institutionalized_male,pop_gpquarters_institutionalized_female,pop_gpquarters_noninst,pop_gpquarters_noninst_male,pop_gpquarters_noninst_female,households,households_married,households_married_withchild,households_cohabit,households_cohabit_withchild,households_singlemale,households_singlemale_alone,households_singlemale_alone_65plus,households_singlemale_withchild,households_singlefemale,households_singlefemale_alone,households_singlefemale_alone_65plus,households_singlefemale_withchild,households_withunder18,households_with65plus,housing,housing_occupied,housing_vacant,housing_vacant_forrent,housing_vacant_rentedunocc,housing_vacant_forsale,housing_vacant_soldunocc,housing_vacant_seasonal,housing_vacant_other,vacancyrate_homeowner,vacancyrate_rental,housing_owneroccupied,housing_renteroccupied
GEOID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36005028300,2084,117,167,130,111,136,159,179,153,139,118,110,147,127,93,75,65,37,21,1651,1606,1543,372,291,916,61,89,61,57,68,68,87,63,54,51,44,69,50,34,27,14,10,9,694,668,635,124,94,1168,56,78,69,54,68,91,92,90,85,67,66,78,77,59,48,51,27,12,957,938,908,248,197,36.2,33.2,38.9,1677,289,517,41,53,0,777,407,2540,567,695,67,65,8,1138,1432,652,148,131,31,8,0,742,372,141,386,10,45,0,35,35,2077,844,174,4,56,9,679,402,66,50,193,52,7,0,0,0,7,6,1,846,177,97,65,33,165,107,26,11,439,144,66,123,317,221,866,846,20,14,0,1,0,0,5,2.0,1.7,49,797
36047086400,2499,108,135,128,159,180,153,172,174,153,171,205,164,181,109,130,61,49,67,2092,2026,1947,527,416,1097,63,77,55,72,90,63,95,77,60,73,90,61,67,48,53,19,15,19,888,857,818,188,154,1402,45,58,73,87,90,90,77,97,93,98,115,103,114,61,77,42,34,48,1204,1169,1129,339,262,41.3,36.8,44.6,2283,60,2169,4,18,0,32,216,2735,92,2371,23,37,2,210,148,2351,15,47,0,0,0,19,67,45,2122,4,18,0,13,149,2499,839,242,5,32,1,725,310,157,103,333,165,0,0,0,0,0,0,0,843,249,98,32,12,175,91,28,20,387,110,

In [4]:
decennial2020_dp_clean.shape

(2292, 153)

In [ ]:
# Remove NYC tracts that are not in the DP or are unpopulated
gdf = preprocessing.remove_unpopulated_tracts()

In [7]:
gdf.shape

(2292, 6)